## Schedule a pipeline run with scheduler API
You can schedule one-time or recurring pipeline runs in Vertex AI using the scheduler API. This lets you implement continuous training in your project.

After you create a schedule, it can have one of the following states:
- `ACTIVE`: An active schedule continuously creates pipeline runs according to the frequency configured using the cron schedule expression. A schedule becomes active on its start time and remains in that state until the specified end time, or until you pause it.
- `PAUSED`: A paused schedule doesn't create pipeline runs. You can resume a paused schedule to make it active again. When you resume a paused schedule, you can use the catch_up parameter to specify whether skipped runs (runs that would have been scheduled if the schedule had been active) need to be rescheduled and submitted at the earliest possible schedule.
- `COMPLETED`: A completed schedule no longer creates new pipeline runs. A schedule is completed according to its specified end time.

Before you schedule a pipeline run using the scheduler API, use the following instructions to set up your Google Cloud project and development environment in the Google Cloud console.
1. Grant the at least one of the following IAM permissions to the user or service account for using the scheduler API:
    - `roles/aiplatform.admin`
    - `roles/aiplatform.user`
2. Build and compile a pipeline.

### Create a schedule
You can create a one-time or recurring schedule.
- Create a schedule based on a `PipelineJob` using the `PipelineJob.create_schedule` method.
- Creating a schedule using the `PipelineJobSchedule.create` method.

While creating a pipeline run schedule, you can also pass the following placeholders supported by the KFP SDK as inputs:
- `{{$.pipeline_job_name_placeholder}}`
- `{{$.pipeline_job_resource_name_placeholder}}`
- `{{$.pipeline_job_id_placeholder}}`
- `{{$.pipeline_task_name_placeholder}}`
- `{{$.pipeline_task_id_placeholder}}`
- `{{$.pipeline_job_create_time_utc_placeholder}}`
- `{{$.pipeline_job_schedule_time_utc_placeholder}}`
- `{{$.pipeline_root_placeholder}}`

#### Create a schedule from a PipelineJob

In [ ]:
from google.cloud import aiplatform

pipeline_job = aiplatform.PipelineJob(
  template_path="COMPILED_PIPELINE_PATH",
  pipeline_root="PIPELINE_ROOT_PATH",
  display_name="DISPLAY_NAME",
)

pipeline_job_schedule = pipeline_job.create_schedule(
    # The name of the pipeline schedule. You can specify a name having a maximum length of 128 UTF-8 characters.
    display_name="SCHEDULE_NAME",
    cron="TZ=CRON",  # Cron schedule expression representing the frequency to schedule and execute pipeline runs
    # The maximum number of concurrent runs for the schedule.
    max_concurrent_run_count=MAX_CONCURRENT_RUN_COUNT,
    # The maximum number of pipeline runs that the schedule creates after which it's completed.
    max_run_count=MAX_RUN_COUNT,
)

#### Create a schedule using PipelineJobSchedule.create

In [ ]:
from google.cloud import aiplatform

pipeline_job = aiplatform.PipelineJob(
    template_path="COMPILED_PIPELINE_PATH",
    pipeline_root="PIPELINE_ROOT_PATH",
    display_name="DISPLAY_NAME",
)

pipeline_job_schedule = aiplatform.PipelineJobSchedule(
    pipeline_job=pipeline_job,
    display_name="SCHEDULE_NAME"
)

pipeline_job_schedule.create(
    cron="TZ=CRON",
    max_concurrent_run_count=MAX_CONCURRENT_RUN_COUNT,
    max_run_count=MAX_RUN_COUNT,
)

### List schedules

In [ ]:
from google.cloud import aiplatform

aiplatform.PipelineJobSchedule.list(
    filter='display_name="DISPLAY_NAME"',
    order_by='create_time desc'
)

### Retrieve a schedule

In [ ]:
from google.cloud import aiplatform

pipeline_job_schedule = aiplatform.PipelineJobSchedule.get(
    schedule_id=SCHEDULE_ID)

### Pause a schedule

In [ ]:
from google.cloud import aiplatform

pipeline_job_schedule = aiplatform.PipelineJobSchedule.get(
    schedule_id=SCHEDULE_ID)

pipeline_job_schedule.pause()

### Update a schedule
When you update a schedule, new runs are scheduled based on the frequency of the updated schedule. New runs are no longer created based on the old schedule and any queued runs are dropped. Pipeline runs that are already created by the old schedule aren't paused or canceled.

In [ ]:
pipeline_job_schedule.update(
    display_name='DISPLAY_NAME',
    max_concurrent_run_count=MAX_CONCURRENT_RUN_COUNT,
)

### Resume a schedule

In [ ]:
pipeline_job_schedule.resume(catch_up=CATCH_UP)

***CATCH_UP***: (Optional) Indicate whether the paused schedule should backfill the skipped pipeline runs. To backfill and reschedule the skipped pipeline runs, enter the following:  
`{ "catch_up":true }`

### Delete a schedule

In [ ]:
pipeline_job_schedule.delete()

### List all pipeline jobs created by a schedule

In [ ]:
pipeline_job_schedule.list_jobs(order_by='create_time_desc')

## Trigger a pipeline run with Cloud Pub/Sub
write, deploy, and trigger a pipeline using an Event-Driven Cloud Function with a Cloud Pub/Sub trigger.

### Build and compile a simple Pipeline

In [ ]:
from kfp import compiler
from kfp import dsl

# A simple component that prints and returns a greeting string
@dsl.component
def hello_world(message: str) -> str:
    greeting_str = f'Hello, {message}'
    print(greeting_str)
    return greeting_str

# A simple pipeline that contains a single hello_world task
@dsl.pipeline(
    name='hello-world-scheduled-pipeline')
def hello_world_scheduled_pipeline(greet_name: str):
    hello_world_task = hello_world(greet_name)


# Compile the pipeline and generate a YAML file
compiler.Compiler().compile(pipeline_func=hello_world_scheduled_pipeline,
                            package_path='hello_world_scheduled_pipeline.yaml')

**Upload compiled pipeline YAML to Cloud Storage bucket**  
Click the uploaded YAML file to access the details. Copy the gsutil URI for later use.

### Create a Cloud Function with Pub/Sub Trigger
Use foolowing `main.py` script for in **Inline Editor** under **Source Code**

In [ ]:
  import base64
  import json
  from google.cloud import aiplatform

  PROJECT_ID = 'your-project-id'                     # <---CHANGE THIS
  REGION = 'your-region'                             # <---CHANGE THIS
  PIPELINE_ROOT = 'your-cloud-storage-pipeline-root' # <---CHANGE THIS

  def subscribe(event, context):
    """Triggered from a message on a Cloud Pub/Sub topic.
    Args:
          event (dict): Event payload.
          context (google.cloud.functions.Context): Metadata for the event.
    """
    # decode the event payload string
    payload_message = base64.b64decode(event['data']).decode('utf-8')
    # parse payload string into JSON object
    payload_json = json.loads(payload_message)
    # trigger pipeline run with payload
    trigger_pipeline_run(payload_json)

  def trigger_pipeline_run(payload_json):
    """Triggers a pipeline run
    Args:
          payload_json: expected in the following format:
            {
              "pipeline_spec_uri": "<path-to-your-compiled-pipeline>",
              "parameter_values": {
                "greet_name": "<any-greet-string>"
              }
            }
    """
    pipeline_spec_uri = payload_json['pipeline_spec_uri']
    parameter_values = payload_json['parameter_values']

    # Create a PipelineJob using the compiled pipeline from pipeline_spec_uri
    aiplatform.init(
        project=PROJECT_ID,
        location=REGION,
    )
    job = aiplatform.PipelineJob(
        display_name='hello-world-pipeline-cloud-function-invocation',
        template_path=pipeline_spec_uri,
        pipeline_root=PIPELINE_ROOT,
        enable_caching=False,
        parameter_values=parameter_values
    )

    # Submit the PipelineJob
    job.submit()

In the `requirements.txt` file, replace the contents with the following package requirements:

In [ ]:
google-api-python-client>=1.7.8,<2
google-cloud-aiplatform

## Send a notification from a pipeline
The following example shows how to configure email notifications by defining an email notification task (`notify_email_task`) and adding it to the pipeline's exit handler (`dsl.ExitHandler`). This notification task invokes the `VertexNotificationEmailOp` operator in the email notification component when the pipeline exits.

In [ ]:
from kfp import dsl
from kfp import compiler
from google_cloud_pipeline_components.v1.vertex_notification_email import VertexNotificationEmailOp

@dsl.pipeline(
    name='PIPELINE_NAME',
    pipeline_root=PIPELINE_ROOT_PATH,
)
def TASK_NAME():    # The name of the pipeline task for which you're configuring email notifications.
    notify_email_task = VertexNotificationEmailOp(recipients=RECIPIENTS_LIST)   # A comma-separated list of up to three email addresses to send the notification email to.

    with dsl.ExitHandler(notify_email_task):
        # Add your pipeline tasks here.

compiler.Compiler().compile(pipeline_func=notification_email_pipeline,
        package_path='notification_email_pipeline.yaml')